In [5]:
import pandas as pd
import numpy as np
from sklearn import svm
import sklearn.preprocessing as skp
# import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

# Basic Code
This part can be ignored, it serves to build the technical process to develop the actual model, but doesn't contain too much to interpret from a model evaluation point of view

In [6]:
#Creates Lagged series
#Goes through a series and generates an lag+1 dimensional pandas DataFrame that has each previous lag timeunit
#as a column and current as the last cobilumn
#Input: Pandas Series
#Output: lag+1 dimensional DataFrame

def timeseriesLagged(data, lag=60):
    df = data
    columns = [df.shift(i) for i in range(1, lag+2)] 
    df = pd.concat(columns,axis=1)
    df.fillna(0, inplace=True)
    df.columns = [str(lag+2-x) for x in range(1,lag+2)]
    # df.reset_index(inplace=True,drop=False)
    df = df[df.columns[::-1]] #Flip because we want newer data on the right
    df= df.iloc[lag+1:] # drop the first 'lag' columns because zeroes.
    df.reset_index(drop=True,inplace=True)
    return df

# Binarizes the last column into 1, 0, -1. 1 = buy 0 = do nothing -1 = sell
# Rate is the percent increase or decrease that should trigger a buy or a sell
# lag is the time unit of lag. 
# atleast is how many of the lookahead need to be atleast the same or greater than flat+rat
# Input: lagged pandas DataFrame, uint lag, double dif, double flat, double atleast between 0 and 1
# Output : Pandas Dataframe with last column binarized
def binarizeTime(resLagged,rate = 0,lookahead = 0, flat = 0,atleast = 0.5):
    if lookahead <= 0 :
        raise Exception("lookahead Must be 1 or higher!")
    resLagged = resLagged.copy() # Make a deep copy
    last = np.shape(resLagged)[1] # find the length of the data 
    last = last-lookahead # convert it to string for loc
    colsLookAhead = list(resLagged.loc[:,str(last+1):str(last + lookahead)])
    colsLast = resLagged[str(last)]
    diffs = resLagged[colsLookAhead].subtract(colsLast,axis=0)
#     print(diffs)
    greater = diffs>=flat  # all the times the price changed higer than flat
    greater = np.count_nonzero(greater,axis=1).reshape((1,-1))
    lesser = diffs<=-flat # all the times the price fell lower than fat
    lesser = np.count_nonzero(lesser,axis=1).reshape((1,-1))
#     return greater,lesser
#     print(greater)
    greater = greater.reshape(1,-1)
    changeToBuy = np.any(greater > lesser & np.greater(greater,atleast*lookahead),axis=0) # make sure more rises than falls and atleast half rises
    changeToSell = np.any(lesser > greater & np.greater(lesser,atleast*lookahead),axis=0)      # make sure more falls than rises and atleast half rises
    changeToHold = ~changeToBuy & ~changeToSell
    resLagged = resLagged.drop(colsLookAhead,1)
    resLagged.loc[changeToSell,str(last+1)] = -1 # Set sell to -1
    resLagged.loc[changeToBuy,str(last+1)] = 1 # Set buy to 1
    resLagged.loc[changeToHold,str(last+1)] = 0 # Set to 0
    return resLagged

In [7]:
# Nifty. Reading and Cleaning
fut = pd.read_csv("Nifty50FUT.csv")
fut['Price']= fut['Price'].str.replace(",","").astype(np.double)
fut['Open']= fut['Open'].str.replace(",","").astype(np.double)
fut['High']= fut['High'].str.replace(",","").astype(np.double)
fut['Low']= fut['Low'].str.replace(",","").astype(np.double)
fut['Vol.']= fut['Vol.'].str.replace(",","").str.replace("M","e6").str.replace("-","0").str.replace("K","e3").astype(np.double)
fut = fut[::-1]
fut.reset_index(inplace=True, drop = True)

nifty = pd.read_csv("Nifty.csv")
nifty = nifty[::-1]
nifty.reset_index(inplace=True, drop = True)
nifty['Price']= nifty['Price'].str.replace(",","").astype(np.double)
nifty['Open']= nifty['Open'].str.replace(",","").astype(np.double)
nifty['High']= nifty['High'].str.replace(",","").astype(np.double)
nifty['Low']= nifty['Low'].str.replace(",","").astype(np.double)
nifty['Vol.']= nifty['Vol.'].str.replace(",","").str.replace("M","e6").str.replace("-","0").str.replace("K","e3").astype(np.double)


niftyDrop = np.setdiff1d(fut['Date'].values,nifty['Date'].values)
futDrop = np.setdiff1d(nifty['Date'].values,fut['Date'].values)
nifty = nifty[~nifty.Date.isin(futDrop)]
fut = fut[~fut.Date.isin(niftyDrop)]

In [8]:
# Nifty Base
volNifty = nifty['Vol.'].diff().dropna()
openNifty = nifty['Open'].diff().dropna()
highNifty = nifty['High'].diff().dropna()
lowNifty = nifty['Low'].diff().dropna()
dataNifty = nifty['Price'].diff().dropna()

# Nifty Futs
volFut = fut['Vol.'].diff().dropna()
openFut = fut['Open'].diff().dropna()
highFut = fut['High'].diff().dropna()
lowFut = fut['Low'].diff().dropna()
dataFut = fut['Price'].diff().dropna()
#Future Premium
prems = dataFut.values - dataNifty.values
prems = pd.Series(prems)

In [9]:
lag=5
lookahead = 1
flat = 0
# First N predicts N+1th. Creating the 1st N series
closeNifty = timeseriesLagged(dataNifty,lag + lookahead-1)
# These are correlated with closeNifty, so we will ignore them for now
# openNifty = timeseriesLagged(openNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
# highNifty = timeseriesLagged(highNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
# lowNifty = timeseriesLagged(lowNifty,lag + lookahead-1).drop(str(lag+1),axis=1)

volNifty = timeseriesLagged(volNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
volNifty = skp.minmax_scale(volNifty,axis=1)

# First N predicts N+1th. Creating the 1st N series
closeFut = timeseriesLagged(dataFut,lag + lookahead-1).drop(str(lag+1),axis=1)
closeFut = skp.minmax_scale(closeFut,axis=1)

# These are correlated with closeNifty, so we will ignore them for now
# openNifty = timeseriesLagged(openNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
# highNifty = timeseriesLagged(highNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
# lowNifty = timeseriesLagged(lowNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
volFut = timeseriesLagged(volFut,lag + lookahead-1).drop(str(lag+1),axis=1)
volFut = skp.minmax_scale(volFut,axis=1)

prems = timeseriesLagged(prems,lag + lookahead-1).drop(str(lag+1),axis=1)
prems = skp.minmax_scale(prems,axis=1)





In [10]:
closeNifty.loc[closeNifty[str(lag+1)] > flat,str(lag+1)] = 1
closeNifty.loc[closeNifty[str(lag+1)] <= flat,str(lag+1)] = 2
closeNifty.loc[closeNifty[str(lag+1)] == flat,str(lag+1)] = 3


In [11]:
n = 5
buySeriesLabs = closeNifty[str(lag+1)] # labels
buySeriesFeats = closeNifty.drop(str(lag+1),axis=1) #features
buySeriesFeats = buySeriesFeats.values
buySeriesFeats = skp.scale(buySeriesFeats,axis=1)

buySeries = np.zeros((len(buySeriesFeats),buySeriesFeats.shape[-1],n))
buySeries[:,:,0] = buySeriesFeats
buySeries[:,:,1] = volFut
buySeries[:,:,2] = prems
buySeries[:,:,3] = closeFut
buySeries[:,:,4] = volNifty



In [12]:
x,y = buySeries,buySeriesLabs
x,y = shuffle(x,y)
tot = len(x)
y = y.values
yOrig = np.copy(y)

x = x.reshape(len(x),-1)
trainPercent = 0.9 # majority of data used for training
testPercent = 0.9 # 
valPercent = 1.00  #

# Test Train Val Split

xTrain = x[0:int(trainPercent*tot),:]
yTrain = y[0:int(trainPercent*tot)]

xTest = x[int(trainPercent*tot): int(testPercent*tot),:]
yTest = y[int(trainPercent*tot): int(testPercent*tot)]

xVal = x[int(testPercent*tot):,:]
yVal = y[int(testPercent*tot):]

In [13]:
clf = svm.SVC(kernel='rbf')
clf.fit(xTrain, yTrain) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
preds = clf.predict(xVal)
print(sum(preds==yVal)/len(yVal))
print(confusion_matrix(yVal,preds))

0.5568181818181818
[[106  46]
 [ 71  41]]


# Actual Dataset
We will try to use Open Interest, Volume, Price from Nifty Futures and Price and Volume from Nifty from the last $LAG$ (variable, will adjust as fit) to make a prediction on whether nifty will increase or decrease as compared to $h$ days ago.

In [551]:
# Read all the data from the csv files
nifty = pd.read_csv("NiftyPrice.csv")
nifty = nifty[::-1]
nifty.reset_index(inplace=True, drop = True)

futOI = pd.read_csv("NiftyFutOI.csv")
futOI = futOI[::-1]
futOI.reset_index(inplace=True, drop = True)

fut = pd.read_csv("NiftyFutPrice.csv")
fut = fut[::-1]
fut.reset_index(inplace=True, drop = True)

oil = pd.read_csv("CrudeOil.csv")
oil = oil[::-1]
oil.reset_index(inplace=True, drop = True)

# Drop any rows not common in both

futDrop = np.setdiff1d(nifty['Date'].values,fut['Date'].values)
niftyDrop = np.setdiff1d(fut['Date'].values,nifty['Date'].values)

nifty = nifty[~nifty.Date.isin(futDrop)]
fut = fut[~fut.Date.isin(niftyDrop)]

In [552]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB

In [553]:
# Create % change daily bars of the different features
# Nifty Base
h = 2 # change from how far ago
volNifty = nifty['PX_VOLUME'].pct_change(h).dropna()*100
dataNifty = nifty['PX_LAST'].pct_change(h).dropna()*100

# Nifty Futs
volFut = fut['PX_VOLUME'].pct_change(h).dropna()*100
oiFut = futOI['OPEN_INT'].pct_change(h).dropna()*100
dataFut = fut['PX_LAST'].pct_change(h).dropna()*100


In [554]:
lag = 12
lookahead = 1
flat = 0
# First N predicts N+1th. Creating the 1st N series
closeNifty = timeseriesLagged(dataNifty,lag + lookahead-1)

volNifty = timeseriesLagged(volNifty,lag + lookahead-1).drop(str(lag+1),axis=1)
volNifty = skp.scale(volNifty,axis=1)

# First N predicts N+1th. Creating the 1st N series
closeFut = timeseriesLagged(dataFut,lag + lookahead-1).drop(str(lag+1),axis=1)
closeFut = skp.scale(closeFut,axis=1)

volFut = timeseriesLagged(volFut,lag + lookahead-1).drop(str(lag+1),axis=1)
volFut = skp.scale(volFut,axis=1)

oiFut = timeseriesLagged(oiFut,lag + lookahead-1).drop(str(lag+1),axis=1)
oiFut = skp.scale(oiFut,axis=1)

In [555]:
# Binazrize the last column
closeNifty.loc[closeNifty[str(lag+1)] > flat,str(lag+1)] = 1
closeNifty.loc[closeNifty[str(lag+1)] <= flat,str(lag+1)] = 2
closeNifty.loc[closeNifty[str(lag+1)] == flat,str(lag+1)] = 3
closeNifty[str(lag+1)]=closeNifty[str(lag+1)].astype(np.int)

In [556]:
# Build the feature vector
n = 5
buySeriesLabs = closeNifty[str(lag+1)] # labels
buySeriesFeats = closeNifty.drop(str(lag+1),axis=1) #features
buySeriesFeats = buySeriesFeats.values
buySeriesFeats = skp.scale(buySeriesFeats,axis=1)

buySeries = np.zeros((len(buySeriesFeats),buySeriesFeats.shape[-1],n))
buySeries[:,:,0] = buySeriesFeats
buySeries[:,:,1] = volFut
buySeries[:,:,2] = oiFut
buySeries[:,:,3] = closeFut
buySeries[:,:,4] = volNifty



In [557]:
# Split into train test val
x,y = buySeries,buySeriesLabs
x,y = shuffle(x,y)
tot = len(x)
y = y.values
yOrig = np.copy(y)

x = x.reshape(len(x),-1)
trainPercent = 0.7 # majority of data used for training
testPercent = 0.7 # 
valPercent = 1.00  #

# Test Train Val Split

xTrain = x[0:int(trainPercent*tot),:]
yTrain = y[0:int(trainPercent*tot)]

xTest = x[int(trainPercent*tot): int(testPercent*tot),:]
yTest = y[int(trainPercent*tot): int(testPercent*tot)]

xVal = x[int(testPercent*tot):,:]
yVal = y[int(testPercent*tot):]

In [558]:
# # Naive Bayes
# nv = GaussianNB()
# nv.fit(x,y)
# preds=nv.predict(xVal)

In [559]:
# sg = SGDClassifier(loss='modified_huber',shuffle=True)
# sg.fit(x,y)
# preds = sg.predict(xVal)

In [560]:
# # Logistic Regression
# lr = LogisticRegression()
# lr.fit(x,y)
# preds=lr.predict(xVal)

In [561]:
# rbf SVM
clf = svm.SVC(kernel='rbf', degree = 2, probability= True) # change kernels between linear, poly and rbf as fit.
clf.fit(xTrain, yTrain) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [562]:
# predsProbs = clf.predict_log_proba(xVal)
# np.argmax(predsProbs,axis=1)

In [568]:
predsProbs = clf.predict_proba(xVal)
preds = clf.predict(xVal)
print("Confusion Matrix")


Confusion Matrix


In [569]:
conf = confusion_matrix(yVal,preds)
tn, fp, fn, tp = conf.ravel()
print(conf)
print("Weight of Buy in data set: %f" % (sum(yVal==1)/len(yVal)))

[[306 105]
 [141 237]]
Weight of Buy in data set: 0.520913


In [570]:
# Precision Recall of Sell
print("Accuracy Percent: %f"% (sum(preds==yVal)/len(yVal)))
print("Precision of Buy Model= %f" % (tp/(tp+fp)))
print("Recall of Buy Model = %f" % (tp/(tp+fn)))

Accuracy Percent: 0.688213
Precision of Buy Model= 0.692982
Recall of Buy Model = 0.626984


In [571]:
# Precision Recall of Sell
print("Accuracy Percent: %f"% (sum(preds==yVal)/len(yVal)))
print("Precision of Sell Model= %f" % (tn/(tn+fn)))
print("Recall of Sell Model = %f" % (tn/(tn+fp)))
print("Weight of Sell in data set: %f" % (sum(yVal==2)/len(yVal)))

Accuracy Percent: 0.688213
Precision of Sell Model= 0.684564
Recall of Sell Model = 0.744526
Weight of Sell in data set: 0.479087


In [ ]:
fn